# Extraxcion de informacion de APIs
Anteriormente calculamos los componentes que tiene mayor correlacion con las companias que entraron en bancarrota, los cuales eran:

Correlacion positiva:
- Debt ratio %
- Current Liability to Assets
- Borrowing dependency (Debt To Equity)
- Current Liability to Current Assets

Correlacion negativa:
- Net Income to Total Assets
- Net worth(totalEquity) / Assets
- Persistent EPS in the Last Four Seasons (EPS)
- Working Capital/total assets


A continuacion utilicemos una API para extraer estas variables de empresas que no se encuentran en nuestro dataset.

In [20]:
import requests
import pandas as pd

## Llave API
Llave gratuita de la API creada en https://site.financialmodelingprep.com/developer ..

In [51]:
API_key = "########################" 

## Obtencion de los tickers de cada empresa
Cada empresa tiene un identificador unico que normalmente esta conformado entre una y cuatro letras mayusculas, obtener los tickers nos costara una llamada por indice, en este caso solo utilizaremos el indice del NASDAQ

In [55]:
# URL de la API para obtener la lista de las empresas listada en el NASDAQ
url_nasdaq = f"https://financialmodelingprep.com/api/v3/nasdaq_constituent?apikey={API_key}"

# Realizar la solicitud GET a la API
response = requests.get(url_nasdaq)
data = response.json()

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    # Crear una lista para almacenar los tickers de las empresas
    tickers = []
    for company in data:
        tickers.append(company['symbol'])
else:
    print("Error al obtener los datos. Código de respuesta:", response.status_code)

Error al obtener los datos. Código de respuesta: 403


Nuestra API es gratuita pero cuenta con un limite de 250 llamadas diarias, para obtener toda la informacion necesaria de cada empresa nuestro codigo "consume" 5 llamadas por empresa. Es por esto que tenemos un maximo de obtencion de informacion de 50 empresas por dia si no se desea pagar un servicio premium.

In [48]:

limite_de_empresas = 40
lista_de_empresas = tickers[0:limite_de_empresas]

## Cuerpo del dataset

In [49]:
columns = ['Debt ratio %','Current Liability to Assets','Borrowing dependency (Debt To Equity)','Current Liability to Current Assets','Net Income to Total Assets','Net worth/Assets','Persistent EPS in the Last Four Seasons (EPS)','Working Capital/total assets']
API_dataset = pd.DataFrame(columns = columns , index = lista_de_empresas)
API_dataset

,Debt ratio %,Current Liability to Assets,Borrowing dependency (Debt To Equity),Current Liability to Current Assets,Net Income to Total Assets,Net worth/Assets,Persistent EPS in the Last Four Seasons (EPS),Working Capital/total assets
ZION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FRC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FLIR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LNC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WDC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CPRT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ALXN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BXP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Extraxcion de datos

In [52]:
for i in range(0,len(API_dataset)):
    
    # Requests 
    ticker = API_dataset.index[i]
    url_ratios = f'https://financialmodelingprep.com/api/v3/ratios-ttm/{ticker}?apikey={API_key}'
    call_ratios = requests.get(url_ratios)
    
    url_balance_sheet = f'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{ticker}?apikey={API_key}'
    call_balance_sheet = requests.get(url_balance_sheet)
    
    url_key_metrics = f'https://financialmodelingprep.com/api/v3/key-metrics-ttm/{ticker}?apikey={API_key}'
    call_key_metrics  = requests.get(url_key_metrics)
    
    url_cash_flow = f'https://financialmodelingprep.com/api/v3/cash-flow-statement/{ticker}?apikey={API_key}'
    call_cash_flow = requests.get(url_cash_flow)
    
    url_income_statement = f'https://financialmodelingprep.com/api/v3/income-statement/{ticker}?apikey={API_key}'
    call_income_statement = requests.get(url_income_statement)
    
    if call_balance_sheet.status_code == 200 and call_ratios.status_code == 200:
        
        # Debt ratio 
        Debt_ratio = call_ratios.json()[0]['debtRatioTTM']
        API_dataset.loc[ticker,'Debt ratio %'] = Debt_ratio

        # Current Liability to Assets
        Current_Liability =  call_balance_sheet.json()[0]['shortTermDebt']
        Total_Assets = call_balance_sheet.json()[0]['totalAssets']
        API_dataset.loc[ticker,'Current Liability to Assets'] =  Current_Liability / Total_Assets
        
        # Borrowing dependency (Debt To Equity)
        Borrowing_dependency = call_key_metrics.json()[0]['debtToEquityTTM']
        API_dataset.loc[ticker,'Borrowing dependency (Debt To Equity)'] = Borrowing_dependency
        
        #Current Liability to Current Assets
        Total_Current_Assets = call_balance_sheet.json()[0]['totalCurrentAssets']
        Current_Liability_to_Current_Asset = Current_Liability / Total_Current_Assets
        API_dataset.loc[ticker,'Current Liability to Current Assets'] = Current_Liability_to_Current_Asset
        
        # Net Income to Total Assets
        Net_income = call_cash_flow.json()[0]['netIncome']
        API_dataset.loc[ticker,'Net Income to Total Assets'] = Net_income / Total_Assets
        
        # Net worth(totalEquity) / Assets
        Net_worth = call_balance_sheet.json()[0]['totalEquity']
        API_dataset.loc[ticker,'Net worth/Assets'] = Net_worth / Total_Assets
        
        # Persistent EPS in the Last Four Seasons (EPS)
        EPS = call_income_statement.json()[0]['eps']
        API_dataset.loc[ticker,'Persistent EPS in the Last Four Seasons (EPS)'] = EPS
        
        # Working Capital/total assets
        Working_capital = call_key_metrics.json()[0]['workingCapitalTTM']
        API_dataset.loc[ticker,'Working Capital/total assets'] = Working_capital/ Total_Assets
    else: 
        print(f"Error al obtener el total assets. Código de estado: {call_balance_sheet.status_code}")
    

In [53]:
API_dataset

,Debt ratio %,Current Liability to Assets,Borrowing dependency (Debt To Equity),Current Liability to Current Assets,Net Income to Total Assets,Net worth/Assets,Persistent EPS in the Last Four Seasons (EPS),Working Capital/total assets
ZION,0.144367,0.07929,2.466628,1.150916,0.010129,0.054643,5.8,0.035613
FRC,0.071854,0.031509,0.875788,1.2568,0.00783,0.082045,8.32,-0.006438
FLIR,0.420919,0.004143,0.726873,0.010969,0.065363,0.579081,1.61,0.242932
LNC,0.018867,0.001496,0.961676,0.004066,-0.006663,0.015266,7.93,0.009772
WDC,0.281221,0.0,8.074201,0.0,0.057123,0.465402,4.81,0.122701
CPRT,0.02036,0.004105,0.023118,0.009895,0.205341,0.871297,2.3,0.469868
ALXN,0.356394,0.007866,0.553746,0.024413,0.033331,0.643606,2.74,0.232464
LB,1.057126,0.051335,-18.505295,0.106471,0.072941,-0.057126,3.04,0.237922
BXP,1.183085,0.576857,4.805374,6.387596,0.04216,0.253346,3.74,0.072522
MAS,0.665378,0.039522,-25.807143,0.069586,0.216503,-0.050511,3.65,0.217852
